In [1]:

import os
os.environ['P8_PG_HOST'] = 'localhost'
os.environ['P8_PG_PORT'] = '35432'
os.environ['P8_PG_PASSWORD'] = os.environ.get('P8_TEST_BEARER_TOKEN')
import nest_asyncio
nest_asyncio.apply()
#print(os.environ.get('P8_TEST_BEARER_TOKEN'))
import percolate as p8
import io
from percolate.services import FileSystemService
fs = FileSystemService()


In [3]:
#data = list(fs.read_chunks('s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf', mode='extended'))
#data = list(fs.read_chunks('/Users/sirsh/Downloads/test_pdf_with_text.pdf', mode='simple'))
data = list(fs.read_chunks('/Users/sirsh/Downloads/INST_017.wav', mode='simple'))

2025-06-10 07:41:36.271 | INFO     | percolate.services.FileSystemService:read_chunks:1226 - Reading and chunking file: /Users/sirsh/Downloads/INST_017.wav (mode: extended)
2025-06-10 07:41:36.275 | INFO     | percolate.services.FileSystemService:read:826 - Reading file: /Users/sirsh/Downloads/INST_017.wav
2025-06-10 07:41:36.278 | INFO     | percolate.services.FileSystemService:read:879 - No specific handler for audio file /Users/sirsh/Downloads/INST_017.wav. Returning raw bytes.
2025-06-10 07:41:36.512 | INFO     | percolate.utils.parsing.ResourceChunker:chunk_resource_from_uri:360 - Chunking resource from URI: /Users/sirsh/Downloads/INST_017.wav (mode: extended)
2025-06-10 07:41:37.183 | INFO     | percolate.services.llm.TranscriptionService:__init__:62 - Using OpenAI client library for transcription
2025-06-10 07:41:37.184 | INFO     | percolate.utils.parsing.ResourceChunker:_chunk_media_resource:498 - Processing audio file: /Users/sirsh/Downloads/INST_017.wav
2025-06-10 07:41:37.8

2025-06-10 07:42:19.516 | INFO     | percolate.services.llm.TranscriptionService:_transcribe_with_client:128 - Preparing to transcribe: /var/folders/tp/cbc5g2n10qgb0lt3tg00dq2h0000gn/T/audio_chunks_dbixlkuh/chunk_3.wav (23.74MB, format: .wav)
2025-06-10 07:42:19.519 | INFO     | percolate.services.llm.TranscriptionService:_transcribe_with_client:138 - WAV file info: channels=1, rate=48000, frames=6224352
2025-06-10 07:42:35.080 | INFO     | percolate.services.llm.TranscriptionService:_transcribe_with_client:152 - Transcription successful: 2026 characters
2025-06-10 07:42:35.083 | INFO     | percolate.utils.parsing.ResourceChunker:_transcribe_audio_chunks:621 - Chunk 3 transcribed: 2026 characters
2025-06-10 07:42:35.086 | INFO     | percolate.utils.parsing.ResourceChunker:_transcribe_audio_chunks:596 - Transcribing audio chunk 4/14
2025-06-10 07:42:35.087 | INFO     | percolate.services.llm.TranscriptionService:transcribe_file:113 - Transcribing file: /var/folders/tp/cbc5g2n10qgb0lt3tg

2025-06-10 07:43:24.122 | INFO     | percolate.services.llm.TranscriptionService:_transcribe_with_client:128 - Preparing to transcribe: /var/folders/tp/cbc5g2n10qgb0lt3tg00dq2h0000gn/T/audio_chunks_dbixlkuh/chunk_9.wav (23.74MB, format: .wav)
2025-06-10 07:43:24.124 | INFO     | percolate.services.llm.TranscriptionService:_transcribe_with_client:138 - WAV file info: channels=1, rate=48000, frames=6224352
2025-06-10 07:43:30.955 | INFO     | percolate.services.llm.TranscriptionService:_transcribe_with_client:152 - Transcription successful: 928 characters
2025-06-10 07:43:30.960 | INFO     | percolate.utils.parsing.ResourceChunker:_transcribe_audio_chunks:621 - Chunk 9 transcribed: 928 characters
2025-06-10 07:43:30.963 | INFO     | percolate.utils.parsing.ResourceChunker:_transcribe_audio_chunks:596 - Transcribing audio chunk 10/14
2025-06-10 07:43:30.964 | INFO     | percolate.services.llm.TranscriptionService:transcribe_file:113 - Transcribing file: /var/folders/tp/cbc5g2n10qgb0lt3tg0

2025-06-10 07:44:16.207 | INFO     | percolate.utils.parsing.ResourceChunker:_create_transcription_resources:667 - Created 10 text chunks from transcription (17233 chars)
2025-06-10 07:44:16.210 | INFO     | percolate.utils.parsing.ResourceChunker:_create_transcription_resources:680 - Created 10 transcription resources for INST_017.wav
2025-06-10 07:44:16.235 | INFO     | percolate.services.FileSystemService:read_chunks:1280 - Successfully created 10 chunks from /Users/sirsh/Downloads/INST_017.wav


In [7]:
from IPython.display import Markdown
Markdown(data[6].content)

o coffees Yeah you didn't have to go we get a million things that's crazy For 40 bucks like Well, it's okay babe babe babe see wait when you're spending the money like that it's a good sign Yeah, like anytime they're telling you to like go out and do something like that like they're just it means that it's like a good client You know what I mean, so it means you're about to like make a bunch of money Hmm It's awesome I got to tell my mom about what you're doing so sweet you have two houses right now that you're like getting like 50% on And they're over a million ones 1.3 ones 1.5 should we do the math right now how much you're gonna make Okay, so So Both of them are 3% and you make basically you make 1.5% And then the 1.5% you make 50% of How did it work last time I think it I think you get like 25% basically

[1037.4s - 1167.1s]: Let's just do a worst case scenario then still it's so you basically get like you get a quarter of 3%. That's what it comes down to because you get half of a half of 3%. So I'm going to do 1,300,000 times 0.03 equals times 0.5 times 0.5. And then, okay, 1,500,000 times 0.3, ah, wait, 1,150,000 times 0.5 equals 0.5 times 0.5. Okay, so babe, at the very, at the very least, you're making 21 grand that's at the least that's, that's like if you do like your split on it and you get 50% commission, no, that's for both added up. It would be, it would be, it would be like way, way more. I'll be right there. I'll be right down mom. No, no, I'll just, I'll be right down. But yeah, it would be so much more if you guys took Anthony's split, obviously it would be double. It would be, it would be double basically. Well, not double because he probably makes like 90% or 80%. But still it would be like 30 grand or like 30, 35 grand.

[1167.1s - 1296.7s]: For both houses? Ok, pretty cool. Yeah, that's perfect. Yeah. That's hard. Um, I don't know. I didn't really think too much into it. Yeah, honestly, it's fire. Wish my split was higher. Yeah, yeah. I'm 55%

In [ ]:
# fs.copy("/Users/sirsh/Downloads/A Programmer’s Guide to the Mind (b).pdf",
#         "s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf")

In [4]:
fs.copy(
        "s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf",
"/Users/sirsh/Downloads/A Programmer’s Guide to the Mind (c).pdf",
)

2025-06-10 06:28:35.221 | INFO     | percolate.services.FileSystemService:copy:919 - Copying file: s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf -> /Users/sirsh/Downloads/A Programmer’s Guide to the Mind (c).pdf
2025-06-10 06:28:35.223 | INFO     | percolate.services.FileSystemService:copy:987 - Using standard read/write copy for s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf -> /Users/sirsh/Downloads/A Programmer’s Guide to the Mind (c).pdf
2025-06-10 06:28:35.397 | INFO     | percolate.services.S3Service:get_streaming_body:1395 - Getting streaming body for s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf
2025-06-10 06:28:35.453 | INFO     | percolate.services.S3Service:get_streaming_body:1400 - Object exists: size=4622893, type=binary/octet-stream
2025-06-10 06:28:35.454 | INFO     | per

In [20]:


#f = fs.open('s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/312eb086-7088-950f-992f-bf724c4cfcf3/A Programmer’s Guide to the Mind (b).pdf', 'rb')
#f = fs.open('/Users/sirsh/Downloads/A Programmer’s Guide to the Mind (c).pdf', 'rb')
f = fs.open('s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf')

#Uploaded with tus (client mode)
f = fs.open("s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/ba5ed262-56a1-b196-61ed-6523235ca144/A Programmer’s Guide to the Mind (c).pdf", 'rb')

#uploaded with tus uppy
f = fs.open('s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/bc7dce83-2a25-8791-6280-76e9b381b7b3/A Programmer’s Guide to the Mind (c).pdf')


2025-06-10 07:23:22.892 | INFO     | percolate.services.S3Service:get_streaming_body:1395 - Getting streaming body for s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf
2025-06-10 07:23:23.073 | INFO     | percolate.services.S3Service:get_streaming_body:1400 - Object exists: size=4622893, type=binary/octet-stream
2025-06-10 07:23:23.074 | INFO     | percolate.services.S3Service:get_streaming_body:1410 - Getting object: bucket=percolate, key=default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/test/A Programmer’s Guide to the Mind (b).pdf
2025-06-10 07:23:23.301 | INFO     | percolate.services.S3Service:get_streaming_body:1413 - Got object response: 4622893 bytes
2025-06-10 07:23:23.732 | INFO     | percolate.services.S3Service:get_streaming_body:1395 - Getting streaming body for s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/ba5ed262-56a1-b196-61ed-6523235ca144/A Programmer’s Guide to the Mind (c).pdf


In [21]:
from pypdf import PdfReader

reader = PdfReader(stream=f)
number_of_pages = len(reader.pages)
page = reader.pages[10]
text = page.extract_text()

text

incorrect startxref pointer(2)
Object 181 0 not defined.
Object 183 0 not defined.
Object 188 0 not defined.
Object 190 0 not defined.
Object 192 0 not defined.
Object 195 0 not defined.
Object 197 0 not defined.
Object 199 0 not defined.
Object 202 0 not defined.
Object 204 0 not defined.
Object 206 0 not defined.
Object 208 0 not defined.
Object 210 0 not defined.
Object 214 0 not defined.
Object 216 0 not defined.
Object 218 0 not defined.
Object 220 0 not defined.
Object 222 0 not defined.
Object 224 0 not defined.
Object 227 0 not defined.
Object 229 0 not defined.
Object 232 0 not defined.
Object 234 0 not defined.
Object 237 0 not defined.
Object 239 0 not defined.
Object 241 0 not defined.
Object 243 0 not defined.
Object 245 0 not defined.
Object 247 0 not defined.
Object 249 0 not defined.
Object 252 0 not defined.
Object 254 0 not defined.
Object 256 0 not defined.
Object 259 0 not defined.
Object 261 0 not defined.
Object 264 0 not defined.
Object 266 0 not defined.
Object 

"Table of Contents       11 \n Knowing through Confidence  128 \nSheltering Confidence  129 \nBuildi ng Confidence  132 \nEmotional 'Knowing'  134 \nKnowing —A Summary and an Analogy  137 \nCulture  139 \nANOTHER LOOK AT CONS CIENCE  143 \nA Digression —Disobedience and Rebellion  143 \nApproval Conscience and Condemnation  146 \nCommon Sense  150 \nNatural Conscience  153 \nHow to Benefit from Conscience  156 \nKNOWING VERSUS `KNOW ING' 160 \nThe Threshold of Uncertainty  160 \nThe Difficulty of Growing Up  165 \nThe Rebellious Teenager  166 \nThe Growing Teenager  169 \nHow to Avoid Teenage Rebellion  171 \nSociety and the Mental Contradiction  176 \nWhat is Me?  180 \nObjective versus Subjective  182 \nApproval and Natural Conscience  185 \nThe Rule of Law  190 "

In [ ]:


fs = FileSystemService()
list(fs.read_chunks('s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/312eb086-7088-950f-992f-bf724c4cfcf3/A Programmer’s Guide to the Mind (b).pdf'))

In [ ]:
from percolate.services import FileSystemService

fs = FileSystemService()
list(fs.read_chunks("/Users/sirsh/Downloads/test_doc.pdf", mode='extended'))

In [ ]:
# from percolate.models import CORE_INSTALL_MODELS,  bootstrap
# #bootstrap(apply=True, apply_to_test_database=False)

# for m in CORE_INSTALL_MODELS:
    
#     p8.repository(m).register()
from percolate.services import FileSystemService

fs = FileSystemService()
g = fs.read_chunks('s3://percolate/default/uploads/10e0a97d-a064-553a-9043-3c1f0a6e6725/ba7358a9-0b05-a16c-1764-fcfa3d491c27/W9SA.pdf',
mode='extended' )

list(g)

In [ ]:
fs.read_chunks?

In [ ]:
import os
import base64
import json 
import uuid

def prepare_metadata(metadata  ) :
    """
    Prepare metadata values for TUS upload.
    
    IMPORTANT: Do not base64 encode values here as TusClient.uploader will
    handle encoding according to the TUS protocol.
    
    Args:
        metadata: Dictionary of metadata key-value pairs
        
    Returns:
        Dictionary with prepared string values ready for TusClient
    """
    prepared = {}
    for key, value in metadata.items():
        # Skip None values completely
        if value is None:
            continue
            
        # Convert complex objects to JSON strings
        if isinstance(value, (dict, list)):
            prepared[key] = json.dumps(value)
        else:
            # Just convert to string, TusClient will handle base64 encoding
            prepared[key] = str(value)
    
    return prepared

DEFAULT_SERVER_URL =  "https://eepis.percolationlabs.ai/tus/"
#DEFAULT_SERVER_URL = "http://localhost:5009/tus/"
# Get server URL from environment or use default
SERVER_URL = os.environ.get("TUS_SERVER_URL", DEFAULT_SERVER_URL)
# Get bearer token from environment
TOKEN = os.environ.get("P8_TEST_BEARER_TOKEN")

# Authentication headers
HEADERS = {"Authorization": f"Bearer {TOKEN}"}
from tusclient.client import TusClient
from tusclient.uploader import Uploader

file_path = "/Users/sirsh/Downloads/INST_017.wav"
filename = os.path.basename(file_path)

# Prepare metadata
metadata = {
    'filename': filename,
    'filetype': 'audio/wav' if filename.endswith('.wav') else 'application/octet-stream',
    'user_id': None,
    'tags': ['test', 'upload', 'percolate']  # Use an array for tags
}

# Get file size
file_size = os.path.getsize(file_path)
chunk_size: int = 5*1024*1024
client = TusClient(SERVER_URL, headers=HEADERS)
    
# Create uploader - using file path directly instead of file stream
uploader = client.uploader(
    file_path=file_path,
    chunk_size=chunk_size,
    metadata=prepare_metadata(metadata),
    retries=3,
    retry_delay=5
)

In [ ]:
filename, file_path

In [ ]:
uploader.upload()
        


In [ ]:
from percolate.services import PostgresService
pg = PostgresService()
pg.execute("""

-- Function to set user context for row-level security
DROP FUNCTION IF EXISTS p8.set_user_context;
CREATE OR REPLACE FUNCTION p8.set_user_context(
    p_user_id UUID, 
    p_role_level INTEGER = NULL
)
RETURNS VOID AS $$
DECLARE
    v_role_level INTEGER;
    v_user_record RECORD;
    v_groups TEXT[];
BEGIN
    -- If role_level not provided, try to load it from the User table
    IF p_role_level IS NULL THEN
        -- Get role_level, required_access_level, and groups from the User table
        SELECT u.role_level, u.required_access_level, u.groups
        INTO v_user_record
        FROM p8."User" u
        WHERE u.id = p_user_id;
        
        -- Use role_level if available, otherwise use required_access_level
        -- If neither is found, default to public access (100)
        IF v_user_record.role_level IS NOT NULL THEN
            v_role_level := v_user_record.role_level;
        ELSIF v_user_record.required_access_level IS NOT NULL THEN
            -- Use the required_access_level as a fallback
            -- This makes sense because God users have required_access_level=0
            -- Admin users have required_access_level=1, etc.
            v_role_level := v_user_record.required_access_level;
        ELSE
            -- Default to public access if nothing is found
            v_role_level := 100;
        END IF;
        
        -- Get user groups if available
        v_groups := v_user_record.groups;
    ELSE
        -- Use the explicitly provided role level
        v_role_level := p_role_level;
        
        -- Still need to get groups from User table
        SELECT u.groups
        INTO v_groups
        FROM p8."User" u
        WHERE u.id = p_user_id;
    END IF;
    
    -- Set the session variables
    PERFORM set_config('percolate.user_id', p_user_id::TEXT, false);
    PERFORM set_config('percolate.role_level', v_role_level::TEXT, false);
    
    -- Set user groups if available
    IF v_groups IS NOT NULL AND array_length(v_groups, 1) > 0 THEN
        -- For LIKE pattern matching in the policy, we need commas as separators
        PERFORM set_config('percolate.user_groups', ',' || array_to_string(v_groups, ',') || ',', false);
    ELSE
        -- Set empty string if no groups
        PERFORM set_config('percolate.user_groups', '', false);
    END IF;
    
    -- Return the role level as a message for debugging
    RAISE NOTICE 'Set user context: user_id=%, role_level=%, groups=%', 
                 p_user_id, v_role_level, v_groups;
END;
$$ LANGUAGE plpgsql SECURITY DEFINER;


""")

In [ ]:
from percolate.api.controllers.tus import process_pending_s3_resources

await process_pending_s3_resources()

In [ ]:
from percolate.models import   Resources

class ExecutiveResources(Resources):
    class Config:
            namespace:str='executive'
            access_level = 1
            description: str = """Executive docs are used as the Resonance data room and may contain confidential information.
    Certain users that have the system admin or executive roles can access these documents.
    We provide information e.g. for due diligence (investors) or to understand Resonance's 
    - Value proposition
    - Mission
    - Values
    - Strategy
    - IP
    etc.
    
    If you dont know how to answer the question from search you can use other resources
    - Entities can be looked up by keys
    - you can recruit the help of other resources by calling the help function and executing the plan
    """
                
import percolate as p8
p8.repository(ExecutiveResources).register()


In [ ]:
import asyncio
import httpx
import os
import json
#print(os.environ.get('P8_TEST_BEARER_TOKEN'))

async def stream_openai_endpoint(url: str):
    headers = {
        "Authorization": f"Bearer {os.environ.get('P8_TEST_BEARER_TOKEN')}",
        "Accept": "text/event-stream",
        "X-User-Email": "sirsh@resonance.nyc",
        "Content-Type": "application/json"
    }

    payload = {
        "model": 'gpt-4.1-mini',
        "messages": [
            {"role": "user", "content": "By seaching for info on Resonance business model using a function, what can you tell me about please be detailed"}
          #  {"role": "user", "content": "what country was i interested in and what did you tell me?"}
           # {"role": "user", "content": "tell me a long poem 10 verses?"}
        ],
        "stream": True
    }
        
    async with httpx.AsyncClient(timeout=None) as client:
        async with client.stream("POST", url, headers=headers, json=payload) as response:
            async for line in response.aiter_lines():
                #print(line)
                if line.startswith("data: "):
                    data = line[len("data: "):]
                    if data.strip() == "[DONE]":
                        print("[DONE]")
                        break
                    try:
                        a = json.loads(data)['choices'][0]['delta']['content']
                        print(a, end='')
                    except:
                        pass
                        print(data)

# Example usage:
import uuid

session_aka_thread_id = str(uuid.uuid1())
#await stream_openai_endpoint(f"http://localhost:5009/v1/agents/executive-ExecutiveDocs/chat/completions?&session_id={session_aka_thread_id}")
await stream_openai_endpoint(f"http://localhost:5009/v1/agents/executive-ExecutiveDocs/chat/completions")
#await stream_openai_endpoint(f"http://p8.resmagic.io/v1/agents/executive-ExecutiveDocs/chat/completions?&session_id={session_aka_thread_id}")


In [ ]:
from percolate.services.llm.CallingContext import get_user_memory

#get_user_memory('4114f279-f345-511b-b375-1953089e078f')

In [ ]:
from percolate.models import AbstractModel, User
import percolate as p8
#p8.repository(User).register()

In [ ]:
from percolate.interface import try_load_model


In [ ]:
M = p8.try_load_model('executive.ExecutiveDocs')

In [ ]:
from IPython.display import Markdown
#Markdown(p8.Agent(M)("If i wanted to scale REsonance in 2026, what do you think, based on the model, would be a good strategy for breaking into new markets"))
#Markdown(p8.Agent(M)("tell me about the network effects that REsonance can levergage to scale different parts of the business"))
Markdown(p8.Agent(M)("If i were a brand, how would i launch with the REsonance platform. Please provide easy to follow steps", audit_on_flush=True))

In [ ]:

p8.repository(M).search("Saoirse Amarteifio")

In [ ]:
import sys
from warnings import filterwarnings
filterwarnings('ignore')
from percolate.models.p8.types import Resources

sys.path.append('/Users/sirsh/code/res/one-core')

In [ ]:
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/INST_018.wav",
    parsing_mode="extended",
    chunk_size=1000,
    chunk_overlap=200,
    save_to_db=False,
    
)

len(resources)

In [ ]:

# Simple chunking - basic text extraction (fast, no API costs)
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/6-Month Roadmap & Milestones.pdf",
    parsing_mode="simple",
    chunk_size=1000,
    chunk_overlap=200,
    save_to_db=False
)

len(resources)

In [ ]:
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/Transit Insurance Form.pdf", 
    parsing_mode="extended",
    save_to_db=True
)
resources

In [ ]:
resources = Resources.chunked_resource(
    uri="/path/to/document.pdf", 
    parsing_mode="extended",
    chunk_size=1000,
    chunk_overlap=200
)

In [ ]:
"""integration test"""

In [ ]:
from one.services.ask_one import AgentCallingContext
from one.services.slack.models import SlackMessage
from percolate.utils import make_uuid
def _get_userid_from_username(name:str):
    """map the user id from the email address or slack name"""
    from percolate.services import PostgresService
    pg = PostgresService()
    Q = f"""
            SELECT * FROM p8."User" where LOWER(email) = LOWER(%s) or LOWER(slack_id) = LOWER(%s) limit 1
    """
    
    data = pg.execute(Q, data=(name, name))
    if data:
        return data[0]['id']
    
    
channel_ts = f"123445"
message = SlackMessage(user='U01JDKSB196', 
                       text='hi, can you tell me thre things about resonance', 
                       thread_ts="123454", 
                       ts=channel_ts, 
                       channel='C08TE6NNY7L')
"""TODO think about this session id interface"""
ctx = AgentCallingContext(
    channel_context=message.channel,
    streaming_callback=None,#,buffer.add_data,
    username=message.user,

    session_id=make_uuid( message.thread_ts or message.ts), #the slack message thread is used as a session id - when there is no message thread_ts its because we are the originator
    channel_ts=str(channel_ts),
    session_context=f"{message.channel}-{message.thread_ts or message.ts}",
    s3_files=None,
    process_files_async=True,
    user_id=_get_userid_from_username(message.user)
    
)
setattr(ctx,'max_iterations',5)

In [ ]:
import percolate as p8
from percolate.models import PercolateAgent

p8.Agent(PercolateAgent).run('what is a fashion brand',context=ctx)

In [ ]:
#!pip install -U boto3

In [ ]:
from percolate.services import S3Service

s =S3Service()
s

In [ ]:
from io import BytesIO
with open("/Users/sirsh/Downloads/6_Proposed Endeavor Guide and Clarifications.pdf", 'rb') as f:
    data = BytesIO(f.read())
    data.seek(0)
    s.upload_filebytes_to_uri("s3://percolate/test/file.pdf", data)

In [ ]:
s.upload_file_to_uri("s3://percolate/test/file.pdf",
                     "/Users/sirsh/Downloads/6_Proposed Endeavor Guide and Clarifications.pdf" )

In [ ]:
import percolate as p8
from percolate.models.media.tus import register_tus_models
from percolate.models import User
#p8.repository(User).register()
#register_tus_models()
from percolate.services import PostgresService

pg = PostgresService()
 

In [ ]:
pg.execute(""" select * from p8."Resources" order by updated_at desc limit 5 """)

In [ ]:
import requests

session = requests.Session()
session.cookies.set('session', 'eyJzeW5jX2ZpbGVzIjogZmFsc2UsICJ0b2tlbiI6IHsiYWNjZXNzX3Rva2VuIjogInlhMjkuYTBBVzRYdHhocFU5Q1ZwU284R2FwR0JQaHZYcHA0S1BJdGREbW9ocmc3ZFNBbllXdEN4ajM3N2pRcTA1Sk03UnhsR3phSk05cWNkU09SWk5rdjlxckdNV1NfUkVqRFhGMVU1Z3FtN0ZDbEhUb3RUM1N3SldyUmJCZGd3d2N2SWlKbWZmSzZDSWZDemdYNHdJZTQ0ZDFlVFV2emkzc1Y4Z2FTTkstX2tENDVhQ2dZS0FXUVNBUmNTRlFIR1gyTWlHLXRJNXlmYzBFTTA5MnFTMnN6aFpnMDE3NSIsICJleHBpcmVzX2luIjogMzU5OSwgInJlZnJlc2hfdG9rZW4iOiAiMS8vMDVZWDNsZG04WXVJeUNnWUlBUkFBR0FVU053Ri1MOUlyYW1wNHlKR2Y5TnA0UW9aUlVGYkdQeVJXaTlWRHZUX0VLdVJ0cFJ3YnJUcHYxS1FVY1VtOFlBV1hQWURBNnhkckc0TSIsICJzY29wZSI6ICJodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL2RyaXZlLnJlYWRvbmx5IGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvdXNlcmluZm8ucHJvZmlsZSBodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL3VzZXJpbmZvLmVtYWlsIGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvZG9jdW1lbnRzLnJlYWRvbmx5IGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvZ21haWwucmVhZG9ubHkgb3BlbmlkIiwgInRva2VuX3R5cGUiOiAiQmVhcmVyIiwgImlkX3Rva2VuIjogImV5SmhiR2NpT2lKU1V6STFOaUlzSW10cFpDSTZJalkyTUdWbU0ySTVOemcwWW1SbU5UWmxZbVU0TlRsbU5UYzNaamRtWWpKbE9HTXhZMlZtWm1JaUxDSjBlWEFpT2lKS1YxUWlmUS5leUpwYzNNaU9pSm9kSFJ3Y3pvdkwyRmpZMjkxYm5SekxtZHZiMmRzWlM1amIyMGlMQ0poZW5BaU9pSXhNRGs1TWpNM09EazBPVE0xTFhBemJqZG5hMncyY3prM2FHb3lNRGh1YjNGdE5ISm9PSEpyTTJkaVluWjBMbUZ3Y0hNdVoyOXZaMnhsZFhObGNtTnZiblJsYm5RdVkyOXRJaXdpWVhWa0lqb2lNVEE1T1RJek56ZzVORGt6TlMxd00yNDNaMnRzTm5NNU4yaHFNakE0Ym05eGJUUnlhRGh5YXpOblltSjJkQzVoY0hCekxtZHZiMmRzWlhWelpYSmpiMjUwWlc1MExtTnZiU0lzSW5OMVlpSTZJakV3TWpFMk5UUTNOakV4TkRnd05EQTVOakl6TUNJc0ltVnRZV2xzSWpvaVlXMWhjblJsZVVCbmJXRnBiQzVqYjIwaUxDSmxiV0ZwYkY5MlpYSnBabWxsWkNJNmRISjFaU3dpWVhSZmFHRnphQ0k2SW5OclYyWktjbTVsTlRaMmEyaFRMVnBKVEhOSlNXY2lMQ0p1YjI1alpTSTZJbkp2WlRNM1UzWkJjbmMyU0VwSVVsQTFTbTl0SWl3aWJtRnRaU0k2SWxOaGIybHljMlVnUVcxaGNuUmxhV1pwYnlJc0luQnBZM1IxY21VaU9pSm9kSFJ3Y3pvdkwyeG9NeTVuYjI5bmJHVjFjMlZ5WTI5dWRHVnVkQzVqYjIwdllTOUJRMmM0YjJOTWFWbFFNM0J0TFhSNmFITldURkp1T1ZoaVoxQlFWa2g0U0RCYWVUQmpSMmROZERKcVlVVkJZMnR3VDBsdlRGUkVaejF6T1RZdFl5SXNJbWRwZG1WdVgyNWhiV1VpT2lKVFlXOXBjbk5sSWl3aVptRnRhV3g1WDI1aGJXVWlPaUpCYldGeWRHVnBabWx2SWl3aWFXRjBJam94TnpRM05EQTJPVGszTENKbGVIQWlPakUzTkRjME1UQTFPVGQ5LmM5NWQ5WGtWREFQcjdjU3hBUWVtZzFYWXROSjREcHJhS2JIYlUwb0t1VmNlcDdybTNxQnRqOTZ4c0VnSWc5SXBWVlQ5Ym4zTDF2alU5OTYyVDdQeC1BWEdNYzRab25YX3FYTGc2M1pueGhBVHNIcXdoQ2VoS2N5NVZxYkxkbVVtYnlsZmFYR0pIbFp5ZDhRbkU1RVppY3B1d3ZhN1U5Y3F2Q184ZTVERkN6Ymk5d0V2bHNmc21Vb3U3U0dWdERkX2dqVUxfNTY4RldhSktrNmt3aUhBZXRUaEQ5WmZUaGgybE1vZHdrVHhuOXVzTWhNZ2oyeENvLXR4bjdqa3hRNnk1NGpINE1DRnhUZzk4V2lRdFJvWkxUbm9neXVYbGZzVkI3V3FQdllqTjlrR2tmMnU3SzJCOHFuMkJGT1NjTFJPaHRGSzJDcTIzVm1WZzhNMmVXQlB6dyIsICJyZWZyZXNoX3Rva2VuX2V4cGlyZXNfaW4iOiA2MDQ3OTksICJleHBpcmVzX2F0IjogMTc0NzQxMDU5NiwgInVzZXJpbmZvIjogeyJpc3MiOiAiaHR0cHM6Ly9hY2NvdW50cy5nb29nbGUuY29tIiwgImF6cCI6ICIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwgImF1ZCI6ICIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwgInN1YiI6ICIxMDIxNjU0NzYxMTQ4MDQwOTYyMzAiLCAiZW1haWwiOiAiYW1hcnRleUBnbWFpbC5jb20iLCAiZW1haWxfdmVyaWZpZWQiOiB0cnVlLCAiYXRfaGFzaCI6ICJza1dmSnJuZTU2dmtoUy1aSUxzSUlnIiwgIm5vbmNlIjogInJvZTM3U3ZBcnc2SEpIUlA1Sm9tIiwgIm5hbWUiOiAiU2FvaXJzZSBBbWFydGVpZmlvIiwgInBpY3R1cmUiOiAiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EvQUNnOG9jTGlZUDNwbS10emhzVkxSbjlYYmdQUFZIeEgwWnkwY0dnTXQyamFFQWNrcE9Jb0xURGc9czk2LWMiLCAiZ2l2ZW5fbmFtZSI6ICJTYW9pcnNlIiwgImZhbWlseV9uYW1lIjogIkFtYXJ0ZWlmaW8iLCAiaWF0IjogMTc0NzQwNjk5NywgImV4cCI6IDE3NDc0MTA1OTd9fSwgInNlc3Npb25faWQiOiAiYzU4ZmQ1MWYtNDAzZC00MjU3LTllMTgtMTc3MzdlNDRmZDNmIn0=.aCdQlQ.qHKMWsxfHuBLHnk3Ck3Ux_tiqp8')

response = session.get('http://localhost:5008/auth/ping')
print(response.json())

response = session.get('http://localhost:5008/auth/session/info')
print(response.json())

In [ ]:
session.get('http://localhost:5008/tus/user/uploads?query=anything').json()

In [ ]:
response = session.get('http://localhost:5008/auth/me')
print(response.json())

In [ ]:
from percolate.api.routes.auth.utils import get_user_from_email, is_valid_token_for_user, extract_user_info_from_token
extract_user_info_from_token('eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6ImlBS2ZoVXRMU1ZDbkR4aVE1X3NwUUEiLCJub25jZSI6IjhGamhCTE5TYzBFeDJ2Qm5KSVA1IiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTk4Mjk0LCJleHAiOjE3NDc2MDE4OTR9.P0goYoml9dxdPHiH-f7IjeEFWMv6qDT6cv-GxHma2drk14pkIuwwdU0w2OCDSlFRGc4hfePIvMU3KIddNPGnkc8qvBKDNvafdBkndMQ0D1rae8BNScVr3EuZiikv_O54Eff1Lv6cmcKd978m5I7p8xVx-2aGGrxGELz3OFszgnh6POo1efiuznp6sPIQJbskfI1a8YR1_GArU9jkwhjhczEsibWNtLn5hIzox02qBTdjfrh69eOOKSmysb2SjWl54AmWxubYfR1M1IO0CAPHUQFzVrdouUsBvZnIk3xWnF-roSMrt68aoWWCtzAxtupzy3ziY8RdTVyWy7uEnr-k6g')

In [ ]:
import requests

session = requests.Session()
#session.cookies.set('session', 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6InNlN2FuOVhHd19BeEVGdkFOSTZSckEiLCJub25jZSI6InZWRlVBekpTU3RsMGZmQnp3RXBsIiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTkzNjk4LCJleHAiOjE3NDc1OTcyOTh9.Z61fPlHZ__DZ38rhlU-PmloMjfs6Tj2xLFqgLj5r2yOHsPDUSEmBYm8Ay7qEmUj5kWV-iyHRgW1c7Z-p3z1k8KUyPVsKXXbZlHvlHUDBmTQo8Uy3axHyei02Vz2ejImBspjlLzjg1bf_wQUXOo5pAdR8D0elaWoRLseslRvyLdZf3k2mk_pxy3c3utkefco1psIaGcdRC44OIdqCCWtPh1MXsSu0fMW4vsdpJQYRrIlmodMWwTLph6Rv2jJ6Rl5JQU3OG6Bx9dW5XQld1k1eQuP2_UEij4-lo3_JsJD4zdbFDSAbiNUbZICof6tatbsDiCpHd3o9-AV_BBlcn_lKdA')

session.cookies.set('session', 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6Im5KcnJIMm9lanZvSHRrSUJyS1FLckEiLCJub25jZSI6Imc1bVk5Z3AyaVduSDF6Q2U2MFl5IiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTk4NTcwLCJleHAiOjE3NDc2MDIxNzB9.NpUvgWfPSpQLaQL8UR1mr2Vd4S7-3ZLEoeK2Dbpbi_uBES51dm8MUFnDTQPNULwo-IVUE9GBAr5HInqTx6L95cTiAohTUG-Qzio96rwtnNYF0YWkBByhTKWRKhdQ6YK97TwGffZMrEzaQ_QvHe_XF17exJpKkRLtjijCuMmmAMMqbdgtsd5XVtxSma_HFapIi4dTcZOiVqWglzsGCXWxvZqtWXiHii1niKFYb95aF8q4sRgsRGs4Z7cCc1l6YzD8sSnkTPyay_qWuAWvMlS3iY9x3777zG82U9Fi38sZ2IaQ7CS1IbZ7CrBo45-2iuh28PSw7H0v2qGcTwfOF0lhTA')

response = session.get('https://eepis.percolationlabs.ai/auth/ping')
print(response.json())

response = session.get('https://eepis.percolationlabs.ai/auth/session/info')
print(response.json())

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?limit=10').json()

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?query=CevolvingAnyibodies.').json()

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?query=Eta Carinae pulsed in the distance.').json()